In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import boxcox, zscore

from create_dataset import ds

# Data cleaning

## Resources

[Becoming One With the Data](https://blog.floydhub.com/becoming-one-with-the-data/)

[sklearn preprocessing documentation](https://scikit-learn.org/stable/modules/preprocessing.html)

[Compare the effect of different scalers on data with outliers - sklearn](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)

[When and Why Should You Normalize / Standardize / Rescale Your Data?](https://medium.com/@swethalakshmanan14/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff)

[Scaling vs Normalization](https://kharshit.github.io/blog/2018/03/23/scaling-vs-normalization)

## What is data

Tables

Images

Text

Temporal

Geospatial

Always remember
- it has to become a number eventually
- machines think in 0 or 1

## What is tabular data

Continuous

Categorical

Ordinal

Binary

Time

## Iterate on data

- not only on models!
- data gathering, labelling, preparation are all iterative
- datasets are often fixed in academia, iterative in industry

![](assets/karpathy.png)

From [Building the Software 2.0 Stack - Andrej Karpathy, Director of AI, Tesla](https://www.youtube.com/watch?v=zywIvINSlaI)

## Characterizing data

Quality = outliers / missing values

Quantity = rows & columns

Diversity = does the distribution match the test set

Cardinality - number of unique values

Dimensionality

Sparsity

Stationarity
- iterating on data
- new / different / more customers
- environment (interest rates changing)
- model predictions infulencing the data (reccomendation, fraud)

Duplicates

Class imbalance

Label noise

Biased sampling

## Sample dataset

A simple dataset:

In [ ]:
ds

## Duplicates

Important to not have duplicates in both your test & train data
- but you should leave duplicate samples if they are only in the train data
- the repetitions provide weight of evidence (especially important in Bayesian methods)
- the frequency infomation is important

Calling `.duplicated()` returns a boolean mask:

In [ ]:
ds.duplicated()

We can use `.duplicated()` to filter out the duplicate:

In [ ]:
ds[~ds.duplicated()]

## Missing values

Not always a `np.Nan` - sometimes this can be encoded as a `0` or a `null` string
- know how missing is represented in your dataset

Always ask - **why is this data missing?**
- is there bias (a pattern) in the missing values?

Dealing with missing values = **detection + replacement**

Detection depends on how the missing value is encoded:

In [ ]:
for col in ds.columns:
    print(col, sum(ds.loc[:, col].isnull()))

### Dropping rows or columns

One solution to missing data is to drop the row (sample) or the column (feature)

- quick & dirty way to deal with missing values
- lose data
- should be done on the first iteration (if working in an agile way)

In [ ]:
ds.dropna(axis=0)

### Imputation for continuous variables

Using a statistic like mean or median:

In [ ]:
data = ds.loc[:, 'contract-length']
filled = data.fillna(data.median())

pd.concat([data, filled], axis=1)

An improvement may be to fill with a statistic that is conditional on another feature:

In [ ]:
data = ds.loc[:, ['contract-length', 'customers-category']]

for customer in set(data.loc[:, 'customers-category']):
    #  get rows with this customer type
    mask = data.loc[:, 'customers-category'] == customer
    
    #  fill the na's using the mean of this customer type
    filled = data.copy()
    filled.loc[:, 'contract-length'].fillna(np.mean(data.loc[mask, 'contract-length']), inplace=True)
    
pd.concat([data, filled], axis=1)

Another solution is to **train a model to predict the missing value**
- perhaps a linear model using other columns as features

### Imputation for categorical variables

Most common class (similar to mean / median)

Missing value token
- tell the model it was missing

Fill with the most common label conditional on another feature (as with continuous)

Train a model to predict it (as with continuous)

# Outliers

Within distribution outliers, versus outliers that are due to mistakes / measurement errors

Require both detection & replacement
- detection is harder than missing values

### Detecting outliers

Using standard deviation
- multiple on the z-score (2 - 4)

In [ ]:
zsc = zscore(ds.loc[:, 'contract-length'].fillna(np.mean(ds.loc[:, 'contract-length'])))
zsc_mask = zsc > 2.0

pd.concat([
    ds, 
    pd.DataFrame({'zsc_mask': zsc_mask, 'zscore': zsc})
], axis=1)

Detection using percentiles
- always lose data 

In [ ]:
#  calculate the 95th percentile
pct = np.percentile(ds.loc[:, 'contract-length'].fillna(np.mean(ds.loc[:, 'contract-length'])), 95)
pct

In [ ]:
#  drop all rows less than this number
ds.loc[ds.loc[:, 'contract-length'] < pct]

Clipping / capping
- this will change the distribution

In [ ]:
max_val = 50
np.clip(ds.loc[:, 'contract-length'], None, max_val)

## String cleaning

Consistent naming
- U.K., UK, United Kingdom, Britian, Great Britian etc

Tokenization
- breaking text into smaller chunks (often splitting on spaces)
- word or sentence level

Normalization
- removing stop words
- lower case
- stemming / lemming

## Scaling

A topic where many people use the various names for transformations in different ways

Scale the validation using the statistics from the training set only
- this is what you have to do in production on single samples!

### Min-max scaling (normalization)

`from sklearn.preprocessing import MinMaxScaler`

Scales to `[0, 1]`
- doesn't shift/center the data
- retains sparsity
- retains zero values

$$ y = \frac{x - x_{min}}{x_{max} - x_{min}} $$

- use when you do not know the distribution of your data
- when you know the distribution is not Gaussian
- algorithm you are using does not make assumptions about the distribution of your data, such as k-nearest neighbors and artificial neural networks
- sensitive to outliers

Use normalization for images

- divide by 255
- pixel values are stored as an 8-bit integer giving a range of possible values from 0 to 255

### Max-abs scaling

`from sklearn.preprocessing import MaxAbsScaler`

Scales to `[-1, 1]`
- divide by largest maximum value

### Standardization

`from sklearn.preprocessing import StandardScaler`

Removing the mean and scaling by the standard deviation

$$ y = \frac{x-\mu}{\sigma} $$

- assumes that your data has a Gaussian distribution
- the algorithm you are using does make assumptions about your data having a Gaussian distribution, such as linear regression, logistic regression, and linear discriminant analysis

### Robust Scalar

`from sklearn.preprocessing import RobustScaler`

$$ y = x - \frac{x_{median}}{x_{IQR}} $$

- subtracting the median to all the observations and then dividing by the interquartile difference. Scales features using statistics that are robust to outliers
- IQR = difference between 75th and 25th percentiles

### Boxcox

Make normally distributed

$$ y = \frac{x - \mu}{x_{max} - x_{min}} $$

- you can do predictions on the transformed data (if your algorithm likes normal looking data), and then inverse-transform back

In [ ]:
raw = np.concatenate([np.random.exponential(size=1000), np.array([10]*20)])

minmax = (raw - np.min(raw)) / (np.max(raw) - np.min(raw))
standard = (raw - np.mean(raw)) / np.var(raw)

f, a = plt.subplots(nrows=4, figsize=(5, 10), sharex=True)

plots = {
    'raw': raw, 'minmax': minmax, 'standardized': standard, 'boxcox': boxcox(raw)
}

for ax, (name, data) in zip(a, plots.items()):
    ax.hist(data)
    ax.set_title(name)

## Fixing target labeling

Most targets have errors/noise.

Sometimes these can be fixed - it depends on the business problem